In [1]:
import itertools as it
import time
import os
import shutil
import textwrap

import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np
import seaborn as sns

import mbuild as mb
import metamds as mds

from mbuild.lib.surfaces import Betacristobalite
from mbuild.lib.atoms import H
from mbuild.lib.moieties import Silane, H2O
from mbuild.examples.pmpc.brush import Brush

used_random_patterns = dict()

In [13]:
def build_monolayer(n_molecules, pattern_class, n_waters, **kwargs):
    if pattern_class is mb.Random2DPattern:
        if n_molecules in used_random_patterns:
            pattern = used_random_patterns[n_molecules]
        else:
            pattern = pattern_class(n_molecules)
        pattern_name = 'rand'
    if pattern_class is mb.Grid2DPattern:
        pattern = pattern_class(int(np.sqrt(n_molecules)),
                                int(np.sqrt(n_molecules)))
        pattern_name = 'grid'

    chain = Brush(1)
    bot = mb.Monolayer(surface=Betacristobalite(),
                       pattern=pattern, 
                       chains=chain,
                       fractions=[0.2],
                       backfill=H())
    mb.translate(bot, [0, 0, 2])

    bot_box = bot.boundingbox
    bot_of_bot = bot_box.mins[2]

    bot_rigid = [i + 1 for i, a in enumerate(bot.particles())
                 if (a.pos[2] < bot_of_bot + 0.05) and a.name == 'Si']
    n_particles = bot.n_particles
    top_rigid = [i + n_particles for i in bot_rigid]

    top = mb.clone(bot)
    mb.spin_y(top, np.pi)
    top_of_bot = bot_box.maxs[2]
    bot_of_top = top.boundingbox.mins[2]
    mb.translate(top, [0, 0, top_of_bot - bot_of_top + 3])
  
    bot_box = bot.boundingbox
    top_box = top.boundingbox
    water_box = mb.Box(mins=[bot_box.mins[0], bot_box.mins[1], bot_box.maxs[2]+0.5],
                       maxs=[bot_box.maxs[0], bot_box.maxs[1], top_box.mins[2]-0.5])
    water = mb.fill_box(H2O(), n_waters, water_box)
    mb.translate(water, [0, 0, top_of_bot + 0.5])

    monolayer = mb.Compound([bot, top, water])
    monolayer.name = 'mpc_n-{}-{}'.format(n_molecules, pattern_name)
    rigid_groups = {'bot': bot_rigid,
                    'top': top_rigid}
    return monolayer, rigid_groups

In [14]:
mon, grps = build_monolayer(n_molecules=20, n_waters=500, pattern_class=mb.Grid2DPattern)

/Users/ctk3b/science/python/mbuild/mbuild/recipes/monolayer.py:83: UserWarning: 
 Adding 16 of chain <Brush 80 particles, non-periodic, 79 bonds, id: 4772959288>
  warnings.warn("\n Adding {} of chain {}".format(len(pattern), chains[-1]))


In [15]:
mon.visualize()

In [3]:
def create_run_script(build_func, forcefield, input_dir, **kwargs):
    compound, rigid_groups = build_func(**kwargs)
    name = compound.name
    em = os.path.join(input_dir, 'em.mdp')
    nvt = os.path.join(input_dir, 'nvt.mdp')
    shear = os.path.join(input_dir, 'const_vel.mdp')
    gro = '{name}.gro'.format(name=name)
    top = '{name}.top'.format(name=name)
    ndx = '{name}.ndx'.format(name=name)

    box = compound.boundingbox
    compound.periodicity += np.array([0, 0, 5 * box.lengths[2]])
    compound.save(top, forcefield=forcefield, overwrite=True)

    with open(ndx, 'w') as f:
        f.write('[ System ]\n')
        atoms = '{}\n'.format(' '.join(str(x + 1) for x in range(compound.n_particles)))
        f.write(textwrap.fill(atoms, 80))
        f.write('\n')
        for name, indices in rigid_groups.items():
            f.write('[ {name} ]\n'.format(name=name))
            atoms = '{}\n'.format(' '.join(str(x) for x in indices))
            f.write(textwrap.fill(atoms, 80))
            f.write('\n')

    cmds = list()
    cmds.append('gmx grompp -f {mdp} -c {gro} -p {top} -n {ndx} -o em.tpr'.format(mdp=em, gro=gro, top=top, ndx=ndx))
    cmds.append('gmx mdrun -v -deffnm em -ntmpi 1')

    cmds.append('gmx grompp -f {mdp} -c em.gro -p {top} -n {ndx} -o nvt.tpr'.format(mdp=nvt, top=top, ndx=ndx))
    cmds.append('gmx mdrun -v -deffnm nvt -ntmpi 1')

    cmds.append('gmx grompp -f {mdp} -c nvt.gro -p {top} -n {ndx} -o shear.tpr'.format(mdp=shear, top=top, ndx=ndx))
    cmds.append('gmx mdrun -v -deffnm shear -ntmpi 1')

    # add shearing commands

    return cmds

In [ ]:
# Initialize a simulation instance with a template and some metadata
try:
    shutil.rmtree('output')
except FileNotFoundError:
    pass
sim = mds.Simulation(name='monolayer', template=create_run_script, output_dir='output')

n_chains = [20]
n_waters = [200]
pattern = mb.Grid2DPattern
for n_mols, n_wats in it.product(n_molecules, n_waters):
    parameters = {'n_chains': n_mols,
                  'n_waters': n_wats,
                  'forcefield': 'OPLS-aa',
                  'pattern_class': pattern,
                  'build_func': build_monolayer}

    # Parameterize our simulation template
    sim.parametrize(**parameters)

# Run
# sim.execute_all(hostname='rahman.vuse.vanderbilt.edu', username='ctk3b')
sim.execute_all()